- WAP to create a UDP server and client in which  client sends a number to which the server sends the cube of the number

In [1]:
%%writefile cubeserver.py

import socket
import time

host = 'localhost'
port = 5000

s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

s.bind((host, port))

print('Server Running.....')

try:
    msg, addr = s.recvfrom(1024)

    received_num_str = msg.decode('utf-8')
    print(f'Received message from {addr} - {received_num_str}')

    num = float(received_num_str)
    
    ans = num ** 3

    s.sendto(str(ans).encode('utf-8'), addr)

except ValueError:
    print("Error: Received message is not a valid number.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    s.close()
    print("Server closed.")

In [2]:
%%writefile cubeclient.py

import socket
import time

clientserver = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
serveraddr = ('localhost', 5000)

number_str = input('Enter the number to send to server - ')

try:
    
    clientserver.sendto(number_str.encode('utf-8'), serveraddr)

    response_bytes, addr = clientserver.recvfrom(1024)

    response_str = response_bytes.decode('utf-8')
    print('Server Message - ', response_str)

except ValueError:
    print("Error: Server response is not a valid number.")
except ConnectionResetError:
    print("ConnectionResetError: The server closed the connection unexpectedly.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")
finally:
    clientserver.close()
    print("Client closed.")

Overwriting cubeclient.py


In [17]:
%%writefile filterclient.py
import socket
HOST='localhost'
PORT=5000
number='1,2,3,4,5'
operation='filter'
message=f'{number}|{operation}'
with socket.socket() as s:
    s.connect((HOST,PORT))
    s.send(message.encode())
    result=s.recv(1024).decode()
    print('Reduced result from server is - ',result)

Overwriting filterclient.py


In [21]:
%%writefile filterserver.py
import socket
from functools import reduce
HOST='localhost'
PORT=5000
def handle_request(number_str,operation):
    numbers=list(map(int,number_str.split(',')))
    if operation== 'map':
        processed=map(lambda x:x*2,numbers)
    elif operation== 'filter':
        processed=filter(lambda x:x%2==0,numbers)
    else:
        return "Invalid Operation"
    reduced_result=reduce(lambda x,y:x+y,processed,0)
    return str(reduced_result)
with socket.socket() as s:
    s.bind((HOST,PORT))
    s.listen()
    print(f'Server listening on {HOST}:{PORT}')
    conn,addr=s.accept()
    with conn:
        print(f'Connected by {addr}')
        data=conn.recv(1024).decode()
        number_str,operation=data.strip().split('|')
        result=handle_request(number_str,operation)
        conn.sendall(result.encode())
    
    

Overwriting filterserver.py


## 344

In [96]:
from bs4 import BeautifulSoup
import pandas as pd

In [97]:
url='Scrape_this.html'
with open(url,"r",encoding='utf-8') as f:
    content=f.read()

In [98]:
# content
soup=BeautifulSoup(content,'html.parser')

In [99]:
tables=soup.find_all('table',class_ = 'common-table')

In [100]:
def extract(table):
    
    #Extract headers
    headers=[i.get_text(strip=True) for i in table.find_all('th')]
    
    #Extract rows
    rows=table.find_all('tr')[1:]
    
    #List table
    tabledata=[]
    
    #Iterate rows
    for row in rows:
        cells=row.find_all('td')
        rowdata=[cell.get_text(strip=True) for cell in cells]
        tabledata.append(rowdata)
        
    return headers,tabledata

In [101]:
header1,data1=extract(tables[0])

In [115]:
dfData=pd.DataFrame(data1,columns=header1)

In [116]:
dfData

,Company Name,Current Price,Change %,Low,High,Value (₹ Cr.),Volume (in 000's)
0,Vodafone Idea,7.75,1.44%,7.33,7.88,35.47,"45,763.00"
1,Reliance Power,15.01,-4.52%,14.82,15.62,31.44,"20,947.33"
2,Yes Bank,16.21,-0.12%,16.15,16.35,16.25,"10,026.87"
3,NHPC,45.80,0.57%,45.62,46.82,23.52,"5,136.20"
4,IDFC First Bank,82.17,1.11%,81.51,84.35,38.62,"4,699.80"
...,...,...,...,...,...,...,...
190,MRF,100429.90,0.70%,99685.70,100900.00,4.73,0.47
191,P&G Hygiene,13961.40,0.31%,13780.90,14012.20,0.52,0.37
192,3M India,27181.25,-2.32%,27039.70,27950.00,0.82,0.30
193,Abbott India,22685.05,-1.69%,22600.05,23249.95,0.64,0.28


In [118]:
from bs4 import BeautifulSoup
import pandas as pd

In [119]:
url='TrumpLies.html'
with open(url,"r",encoding='utf-8') as f:
    content=f.read()

In [120]:
soup=BeautifulSoup(content,'html.parser')

In [145]:
data=soup.find_all('span',class_='short-desc')
DATE=[i.get_text(strip=True).split('“')[0] for i in data]
STATEMENT=[i.get_text(strip=True).split('“')[1] for i in data]
d={'Date':DATE,'Statement':STATEMENT}
df=pd.DataFrame(d)
df

,Date,Statement
0,Jan. 21,I wasn't a fan of Iraq. I didn't want to go in...
1,Jan. 21,A reporter for Time magazine — and I have been...
2,Jan. 23,Between 3 million and 5 million illegal votes ...
3,Jan. 25,"Now, the audience was the biggest ever. But th..."
4,Jan. 25,Take a look at the Pew reports (which show vot...
...,...,...
175,Oct. 25,We have trade deficits with almost everybody.”...
176,Oct. 27,"Wacky & totally unhinged Tom Steyer, who has b..."
177,Nov. 1,"Again, we're the highest-taxed nation, just ab..."
178,Nov. 7,When you look at the city with the strongest g...
